In [35]:
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
import folium
from IPython.display import display
from collections import namedtuple
from pyproj import CRS, Transformer

In [2]:
file_path = r'data\WalkabilityIndex\Natl_WI.gdb'

# Assuming gdf is already defined
gdf = gpd.read_file(file_path)

# Inspect the dataframe
gdf.head(n=1)

,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,CBSA_Name,...,D3B,D4A,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,geometry
0,481130078254,481130078254,48,113,007825,4,206,"Dallas-Fort Worth, TX-OK",19100,"Dallas-Fort Worth-Arlington, TX",...,115.981747,362.1,6.0,14.0,15.0,17.0,14.0,3110.36082,297836.08309,"MULTIPOLYGON (((-68983.316 1091325.734, -68981..."


In [25]:
# Count the GEOID20 per CBSA_Name. Convert to dataframe 'blocks
blocks = gdf.groupby('CBSA_Name')['GEOID20'].count().reset_index()

# Rename the columns
blocks.columns = ['CBSA_Name', 'Count of Blocks']

# Sort the values on 'Count of Blocks
blocks = blocks.sort_values(by='Count of Blocks', ascending=False)
blocks.head()


,CBSA_Name,Count of Blocks
604,"New York-Newark-Jersey City, NY-NJ-PA",14376
498,"Los Angeles-Long Beach-Anaheim, CA",8248
162,"Chicago-Naperville-Elgin, IL-IN-WI",6590
664,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",4306
211,"Dallas-Fort Worth-Arlington, TX",4128


In [37]:
# Define the Location named tuple
Location = namedtuple('Location', ['longitude', 'latitude'])

def load_geodataframe(filepath):
    print(f"Loading GeoDataFrame from {filepath}")
    gdf = gpd.read_file(filepath)
    print(f"GeoDataFrame loaded with {len(gdf)} records")
    
    # Print the original CRS of the GeoDataFrame
    print(f"Original GeoDataFrame CRS: {gdf.crs}")
    
    # Reproject the GeoDataFrame to EPSG:3857
    gdf = gdf.to_crs(epsg=3857)
    print(f"GeoDataFrame reprojected to CRS EPSG:3857")
    
    return gdf

def get_city_location(city_name, user_agent="city_walkability_app"):
    print(f"Getting location for city: {city_name}")
    geolocator = Nominatim(user_agent=user_agent)
    location = geolocator.geocode(city_name)
    
    if location:
        # Assuming geopy returns latitude and longitude in EPSG:4326
        print(f"Location found: {location.latitude}, {location.longitude}")
        
        # Check if location's CRS is EPSG:4326, if not, convert
        location_crs = CRS.from_epsg(4326)  # geopy uses EPSG:4326 by default
        target_crs = CRS.from_epsg(3857)
        
        transformer = Transformer.from_crs(location_crs, target_crs, always_xy=True)
        longitude_3857, latitude_3857 = transformer.transform(location.longitude, location.latitude)
        
        print(f"Location in EPSG:3857: {latitude_3857}, {longitude_3857}")
        
        # Return the transformed location as a named tuple
        return Location(longitude=longitude_3857, latitude=latitude_3857)
    else:
        print("Location not found")
        return None

# .1 degree is approximately 6.9 miles
def filter_geodataframe_by_city(gdf, location, buffer_radius=.1):
    # Ensure the GeoDataFrame has a CRS
    if gdf.crs is None:
        raise ValueError("GeoDataFrame must have a CRS defined.")
    
    # Create a point for the location
    city_point = Point(location.longitude, location.latitude)
    
    # Ensure the point has the same CRS as the GeoDataFrame
    city_point_gdf = gpd.GeoDataFrame([{'geometry': city_point}], crs=gdf.crs)
    
    # Create a buffer around the point
    city_buffer = city_point_gdf.buffer(buffer_radius).iloc[0]
    
    # Filter the GeoDataFrame based on the buffer
    filtered_gdf = gdf[gdf.geometry.intersects(city_buffer)]
    
    print(f"Filtering GeoDataFrame by city buffer with radius {buffer_radius}")
    print(f"Filtered GeoDataFrame contains {len(filtered_gdf)} records")
    
    return filtered_gdf

def simplify_geometries(gdf, tolerance=0.01):
    print(f"Simplifying geometries with tolerance {tolerance}")
    gdf = gdf.copy()  # Create a copy of the GeoDataFrame
    gdf['geometry'] = gdf['geometry'].simplify(tolerance=tolerance, preserve_topology=True)
    print("Geometries simplified")
    return gdf

def display_walkability_index(gdf):
    print("Displaying walkability index for each geometry")
    for index, row in gdf.iterrows():
        print(f"Geometry: {row['geometry']}, NatWalkInd: {row['NatWalkInd']}")

def calculate_memory_usage(gdf):
    print("Calculating memory usage of the GeoDataFrame")
    memory_usage_bytes = gdf.memory_usage(deep=True).sum()
    memory_usage_mb = memory_usage_bytes / (1024 ** 2)
    print(f"Filtered GeoDataFrame size: {memory_usage_mb:.2f} MB")

def create_map(gdf, location):
    print(f"Creating map centered at {location.latitude}, {location.longitude} (EPSG:3857)")
    
    # Transform coordinates from EPSG:3857 to EPSG:4326
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
    lon_4326, lat_4326 = transformer.transform(location.longitude, location.latitude)
    
    print(f"Transformed coordinates to EPSG:4326: {lat_4326}, {lon_4326}")
    
    m = folium.Map(location=[lat_4326, lon_4326], zoom_start=12, width="100%", height="100%")
    
    def style_function(feature):
        return {
            'fillColor': '#0000ff',
            'color': '#ff0000',
            'weight': 3,
            'fillOpacity': 0.5,
            'opacity': 0.8
        }
    
    folium.GeoJson(gdf, style_function=style_function).add_to(m)
    print("Map created")
    return m

def display_map(m):
    print("Displaying map")
    display(m)

def main():
    filepath = r'data\WalkabilityIndex\Natl_WI.gdb'
    city_name = "Dallas"
    
    gdf = load_geodataframe(filepath)
    location = get_city_location(city_name)
    
    if location:
        city_gdf = filter_geodataframe_by_city(gdf, location)
        city_gdf = simplify_geometries(city_gdf)
        display_walkability_index(city_gdf)
        calculate_memory_usage(city_gdf)
        m = create_map(city_gdf, location)
        display_map(m)
    else:
        print("City not found. Please enter a valid city name.")

# Run the main function
main()

Loading GeoDataFrame from data\WalkabilityIndex\Natl_WI.gdb
GeoDataFrame loaded with 220739 records
Original GeoDataFrame CRS: PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102039"]]
GeoDataFrame reprojected to CRS EPSG:3857
Getting location for city: Dallas
Location found: 32.7762719, -96.7968559
Location in EPSG:3857: 3865645.31309221, -10775376.709177878
Fi

In [31]:
# import geopandas as gpd
# from shapely.geometry import Point

# def filter_geodataframe_by_city(gdf, location, buffer_radius=0.1):
#     # Ensure the GeoDataFrame has a CRS
#     if gdf.crs is None:
#         raise ValueError("GeoDataFrame must have a CRS defined.")
    
#     # Print the original CRS of the GeoDataFrame
#     print(f"Original GeoDataFrame CRS: {gdf.crs}\n")
    
#     # Reproject the GeoDataFrame to EPSG:3857
#     gdf_projected = gdf.to_crs(epsg=3857)
    
#     # Print the projected GeoDataFrame CRS
#     print(f"Projected GeoDataFrame CRS: {gdf_projected.crs}\n")
    
#     # Create a point for the location in EPSG:4326
#     city_point = Point(location.longitude, location.latitude)
    
#     # Create a GeoDataFrame for the point with the CRS EPSG:4326
#     city_point_gdf = gpd.GeoDataFrame([{'geometry': city_point}], crs="EPSG:4326")
    
#     # Reproject the point GeoDataFrame to EPSG:3857
#     city_point_gdf_projected = city_point_gdf.to_crs(epsg=3857)
    
#     # Print the point GeoDataFrame
#     print(f"City Point GeoDataFrame (Projected): {city_point_gdf_projected}\n")
    
#     # Create a buffer around the point in the projected CRS
#     city_buffer = city_point_gdf_projected.buffer(buffer_radius * 1609.34).iloc[0]  # Convert miles to meters
    
#     # Print the buffer
#     print(f"City Buffer: {city_buffer}\n")
    
#     # Filter the GeoDataFrame based on the buffer
#     filtered_gdf = gdf_projected[gdf_projected.geometry.intersects(city_buffer)]
    
#     print(f"Filtering GeoDataFrame by city buffer with radius {buffer_radius} miles\n")
#     print(f"Filtered GeoDataFrame contains {len(filtered_gdf)} records\n")
    
#     return filtered_gdf

# # Example usage:
# from collections import namedtuple

# # Define the Location named tuple
# Location = namedtuple('Location', ['longitude', 'latitude'])

# # Create an instance of Location with the given coordinates
# location = Location(longitude=-96.7968559, latitude=32.7762719)

# # Call the function with the GeoDataFrame, Location instance, and buffer radius
# filtered_gdf = filter_geodataframe_by_city(gdf, location, buffer_radius=0.1)

# # Print the filtered GeoDataFrame
# print(filtered_gdf)

Original GeoDataFrame CRS: PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","102039"]]

Projected GeoDataFrame CRS: EPSG:3857

City Point GeoDataFrame (Projected):                             geometry
0  POINT (-10775376.709 3865645.313)

City Buffer: POLYGON ((-10775215.775177877 3865645.31309221, -10775216.550119076 3865629.538801748, -10775218.86747956 3865613.